In [6]:
"""
Arjun Srivastava
arj1
AMATH 301 B
"""

import numpy as np
import scipy.linalg
import pandas as pd
import time

In [7]:
# Problem 1

# a)

a, b = 1e21 - (1e21 - 1e5), (1e21 - 1e21) + 1e5

diff = abs(a-b)  # diff = 31072.0

"""
Considering the size of the numbers a and b, I would initially say that this result is not very significant. 31072.0, however,
is still a large number, and there are many contexts in which this error would be significant.
"""

# b)

A = np.array([[10e-20, 1], [1, 1]])

cond = np.linalg.cond(A)  # cond = 2.6180339887498953, which implies a unique solution

# c)

L, U = np.array([[1, 0], [10e20, 1]]), np.array([[10e-20, 1], [0, 1-10e20]])
print('='*30, 'c)', sep='\n')
print(A, (L @ U), sep='\n\n')

"""
L @ U is not equal to A. The first row is, and this is because we are not adding values that involve catastrophic cancellation.
The bottom row, however, involves 10e20 + 10e-20, which should be equal to 0. As can be seen by the result printed below,
there is an extra 100. This makes sense due to the immense size of 10e20 and 10e-20. The second column of the second row
has 10e20 - 10e20. The result should be 1, but it is instead 0. Catastrophic cancellation is occurring in both of these cases
since 10e20 and 10e-20 are being operated on together.
"""

# d)

B = np.array([[1, 1], [10e-20, 1]])
L1, U1 = np.array([[1, 0], [10e-20, 1]]), np.array([[1, 1], [0, 1-10e-20]])
print('='*30, 'd)', sep='\n')
print(B, (L1 @ U1), sep='\n\n')
print('='*30, 'e)', sep='\n')

"""
By reordering the matrix A to B, we have avoided any operation that involves two very large numbers (10e20, 10e-20). Since
10e20 and 10e-20 are only ever being added to or subtracted by 1, there is no case in which catastrophic cancellation occurs,
and our answers are much more accurate.
"""

# e)

P, L2, U2 = scipy.linalg.lu(A)
P = P.T
print(L2, U2, P, sep='\n')
print('\n')

P1, L3, U3 = scipy.linalg.lu(B)
P1 = P1.T
print(L3, U3, P1, sep='\n')

"""
The official SciPy LU yields the same results as part d). This implies that Python is using the permutation matrix
to ensure that the matrix A can be more efficiently decomposed without issues. As shown, P is different for A and B,
implying that Python wants to avoid catastrophic cancellation by setting up the equation in a more efficient manner. Since
A is not set up efficiently for an LU decomp, the permutation matrix converts it.
"""

c)
[[1.e-19 1.e+00]
 [1.e+00 1.e+00]]

[[1.e-19 1.e+00]
 [1.e+02 0.e+00]]
d)
[[1.e+00 1.e+00]
 [1.e-19 1.e+00]]

[[1.e+00 1.e+00]
 [1.e-19 1.e+00]]
e)
[[1.e+00 0.e+00]
 [1.e-19 1.e+00]]
[[1. 1.]
 [0. 1.]]
[[0. 1.]
 [1. 0.]]


[[1.e+00 0.e+00]
 [1.e-19 1.e+00]]
[[1. 1.]
 [0. 1.]]
[[1. 0.]
 [0. 1.]]


'\nThe official SciPy LU yields the same results as part d). This implies that Python is using the permutation matrix\nto ensure that the matrix A can be more efficiently decomposed without issues. As shown, P is different for A and B,\nimplying that Python wants to avoid catastrophic cancellation by setting up the equation in a more efficient manner. Since\nA is not set up efficiently for an LU decomp, the permutation matrix converts it.\n'

In [8]:
# Problem 2

# a)

A = np.genfromtxt('example_matrix.csv', delimiter=',')

In [30]:
# b)

t0 = time.time()
rb = []
for i in range(100):
    b = np.random.rand(3000 , 1)
    x = scipy.linalg.solve(A, b)
    rb.append((A @ x) - b)
t1 = time.time()
final_time_b = t1 - t0  # 44.55109429359436 seconds
rb = np.max(np.abs(rb))  # 1.3355982986240633e-13

In [31]:
# c)

t0 = time.time()
P, L, U = scipy.linalg.lu(A)
P = P.T
rc = []
for i in range(100):
    b = np.random.rand(3000, 1)
    y = scipy.linalg.solve_triangular(L, P @ b, lower=True)
    x = scipy.linalg.solve_triangular(U, y)
    rc.append((A @ x) - b)
t1 = time.time()
final_time_c = t1 - t0  # 4.096189737319946 seconds
rc = np.max(np.abs(rc))  # 1.6930901125533637e-13

In [32]:
# d)

t0 = time.time()
inv = scipy.linalg.inv(A)
rd = []
for i in range(100):
    b = np.random.rand(3000, 1)
    x = inv @ b
    rd.append((A @ x) - b)
t1 = time.time()
final_time_d = t1 - t0  # 1.173656940460205 seconds
rd = np.max(np.abs(rd))  # 2.9465319073551655e-13

# e) modified the code above ^

In [33]:
# f)

data = {'Method': ['G.E.', 'LU', 'Inverse'], 'total time': [final_time_b, final_time_c, final_time_d], 'average residual': [rb, rc, rd]}
table = pd.DataFrame(data)

# g)

"""
If I only cared about speed, I would choose the inverse method, as it is significantly faster than the Gaussian Elimination method
and fairly quicker than the LU decomposition. The LU decomposition is close behind (3 seconds slower).
"""

# h)

"""
If I only cared about accuracy, I would choose the reliable Gaussian Elimination method, as it has the lowest maximum residual.
The LU decomposition has a residual that is barely higher, however, and it is much more time efficient.
"""

# i)

"""
The most ideal method seems like the LU decomposition. It has a fast time and much less residual error than the inverse method.
Overall, it seems like the best tradeoff for guaranteeing fast and accurate results.
"""

# j)

"""
The times are increased if these calculations are done inside the loop. For example, the inverse loop took 1.17 seconds before,
but now takes 65.18 seconds. This is a very important detail, especially if we are trying to analyze the runtime of a specific
function or process. The LU decomposition and the inverse calculations all take time, and we should always put pre and post
steps outside of the time calculation. In the inverse case, the time increase was extremely significant, proving that all steps
matter.
"""

# k)

"""
To better time the efficiency of the solve function, I would start and end my times() within the for loop itself, starting
before the solve function and ending right after. After the time is closed for that particular iteration of the loop,
I would add the difference between the ending time and starting time to an int (which was defined before the loop), and then
check the value of the int after the for loop is done. This way, I am only spending time calculating the solve function, and not
the creation of the range() object or the creation of the random vector. If I were to implement this, I am sure the times would be smaller
across the board for each method.
"""

table  # show DataFrame

,Method,total time,average residual
0,G.E.,46.707110,1.755263e-13
1,LU,4.780220,1.696421e-13
2,Inverse,1.593739,2.484679e-13
